In [12]:
import torch, torchtext, numpy as np
import pandas as pd, csv
from torch import nn, optim
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pdb
from keras.models import Sequential
from tensorflow import keras
from scipy import signal
import os

In [15]:
# assign directory
directory = "drive/MyDrive/MINT_COLOR_DATA"
 
# iterate over files in
# that directory
dataset=[]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        ds = pd.read_csv(f)
        print(f)
        i=ds.loc[ds[' EXG Channel 0'] == 0].index
        if(len(i)==1):
          ds=ds.tail(-(i[0]+1))
        dataset.append(ds)

drive/MyDrive/MINT_COLOR_DATA/OpenBCI-RAW-2022-10-21_18-52-18.csv
drive/MyDrive/MINT_COLOR_DATA/OpenBCI-RAW-2022-10-21_18-51-56.csv
drive/MyDrive/MINT_COLOR_DATA/OpenBCI-RAW-2022-10-21_18-51-33.csv
drive/MyDrive/MINT_COLOR_DATA/OpenBCI-RAW-2022-10-21_18-51-10.csv
drive/MyDrive/MINT_COLOR_DATA/OpenBCI-RAW-2022-10-21_18-50-47.csv
drive/MyDrive/MINT_COLOR_DATA/OpenBCI-RAW-2022-10-21_18-50-23.csv
drive/MyDrive/MINT_COLOR_DATA/OpenBCI-RAW-2022-10-21_18-49-57.csv
drive/MyDrive/MINT_COLOR_DATA/OpenBCI-RAW-2022-10-21_18-49-16.csv
drive/MyDrive/MINT_COLOR_DATA/OpenBCI-RAW-2022-10-21_18-48-52.csv
drive/MyDrive/MINT_COLOR_DATA/OpenBCI-RAW-2022-10-21_18-52-41.csv
drive/MyDrive/MINT_COLOR_DATA/OpenBCI-RAW-2022-10-23_13-15-10.csv
drive/MyDrive/MINT_COLOR_DATA/OpenBCI-RAW-2022-10-23_13-14-46.csv
drive/MyDrive/MINT_COLOR_DATA/OpenBCI-RAW-2022-10-23_13-14-23.csv
drive/MyDrive/MINT_COLOR_DATA/OpenBCI-RAW-2022-10-23_13-13-54.csv
drive/MyDrive/MINT_COLOR_DATA/OpenBCI-RAW-2022-10-21_18-55-51.csv
drive/MyDr

In [16]:
dataset[8]

,Sample Index,EXG Channel 0,EXG Channel 1,EXG Channel 2,EXG Channel 3,EXG Channel 4,EXG Channel 5,EXG Channel 6,EXG Channel 7,Accel Channel 0,...,Other.3,Other.4,Other.5,Other.6,Analog Channel 0,Analog Channel 1,Analog Channel 2,Timestamp,Other.7,Timestamp (Formatted)
123,1.0,-104868.439420,-141932.593814,-32949.265295,-114339.678209,22512.207629,-72299.153781,1822.024801,-131203.264738,0.066,...,30.0,16.0,0.0,16.0,0.0,0.0,0.0,1.666403e+09,0.0,2022-10-21 18:48:52.789
124,2.0,-104859.185798,-141923.183730,-32941.486888,-114339.655857,22522.221210,-72307.714499,1821.331897,-131195.553386,0.066,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666403e+09,0.0,2022-10-21 18:48:53.268
125,3.0,-104861.286862,-141927.095285,-32947.723025,-114340.482872,22518.577876,-72315.805830,1814.693429,-131196.000421,0.066,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666403e+09,0.0,2022-10-21 18:48:53.268
126,4.0,-104862.069173,-141927.922300,-32951.880449,-114337.040703,22512.967588,-72310.597874,1834.430019,-131194.704019,0.066,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666403e+09,0.0,2022-10-21 18:48:53.268
127,5.0,-104862.940891,-141930.537454,-32951.031083,-114334.582011,22519.047263,-72299.667871,1863.643749,-131188.669048,0.066,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666403e+09,0.0,2022-10-21 18:48:53.268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5147,161.0,-106394.817697,-143213.818158,-41041.580265,-116876.668260,24814.817287,-77453.756625,7755.362422,-129829.928855,0.110,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666403e+09,0.0,2022-10-21 18:49:12.936
5148,162.0,-106392.828392,-143210.912432,-41042.966073,-116887.732373,24815.040805,-77467.502948,7745.102971,-129830.018262,0.106,...,29.0,176.0,4.0,48.0,0.0,0.0,0.0,1.666403e+09,0.0,2022-10-21 18:49:12.936
5149,163.0,-106387.732194,-143204.788054,-41038.026337,-116874.075457,24821.343997,-77455.298895,7757.977576,-129820.429363,0.106,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666403e+09,0.0,2022-10-21 18:49:12.936
5150,164.0,-106390.526162,-143209.683086,-41041.401451,-116874.455437,24818.281808,-77458.897526,7770.047518,-129826.933721,0.106,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666403e+09,0.0,2022-10-21 18:49:12.936


In [17]:
for data in dataset:
  data.drop(data.iloc[:, 9:25], axis=1, inplace=True)
dataset[8].shape

(5029, 9)

In [18]:
i =0;
for data in dataset:
  if data.shape[0] < 4352:
    i=i+1
i

0

In [19]:
data_train=dataset[0:40]
data_test=dataset[40:52]

In [20]:
from sklearn.preprocessing import MinMaxScaler
scaler =MinMaxScaler()
i=0;
for data in dataset[0:40]:
  data_train[i] = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)
  i=i+1
i=0
for data in dataset[40:52]:
  data_test[i] = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)
  i=i+1

In [41]:
import tqdm #0 is green, 1 is blue, 2 is red
n=256*5
feature_names=list(data_train[0].drop('Sample Index',axis=1).columns)
X=[]
Y=[]
indexes=[]
for data in data_train:
  norm_data_x =data[feature_names]
  for i in tqdm.tqdm_notebook(range(3)):
    X.append(norm_data_x.iloc[i*n+i*256:i*n+n+i*256].  values)
    Y.append(i)
    indexes.append(data.index[i+n-1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [57]:
print(X[119].shape)
print(Y[119])

(1280, 8)
2


In [61]:
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.optimizers import adam_v2
n_steps=256*5
n_features=8
model=Sequential()
model.add(LSTM(100, activation='relu', return_sequences=False, input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(Dense(1))
opt = adam_v2.Adam(lr=0.0005, decay=1e-6)

model.compile(optimizer=opt, loss='mse', metrics=['mape'])

In [62]:
history=model.fit(np.asarray(X),np.asarray(Y),batch_size=20,epochs=15)

Epoch 1/15
6/6 [==============================] - 5s 582ms/step - loss: 1.2909 - mape: 69175632.0000
Epoch 2/15
6/6 [==============================] - 4s 582ms/step - loss: 0.9804 - mape: 170248240.0000
Epoch 3/15
6/6 [==============================] - 3s 584ms/step - loss: 0.6971 - mape: 252516752.0000
Epoch 4/15
6/6 [==============================] - 4s 586ms/step - loss: 0.5967 - mape: 338491264.0000
Epoch 5/15
6/6 [==============================] - 4s 595ms/step - loss: 0.4460 - mape: 286703456.0000
Epoch 6/15
6/6 [==============================] - 4s 588ms/step - loss: 0.3721 - mape: 233047040.0000
Epoch 7/15
6/6 [==============================] - 3s 574ms/step - loss: 0.3219 - mape: 179928944.0000
Epoch 8/15
6/6 [==============================] - 4s 600ms/step - loss: 0.2292 - mape: 162490208.0000
Epoch 9/15
6/6 [==============================] - 4s 586ms/step - loss: 0.2116 - mape: 124670024.0000
Epoch 10/15
6/6 [==============================] - 4s 603ms/step - loss: 0.2778 - m

In [63]:
n=256*5
feature_names=list(data_test[0].drop('Sample Index',axis=1).columns)
X_test=[]
Y_test=[]
indexes=[]
for data in data_test:
  norm_data_x =data[feature_names]
  for i in tqdm.tqdm_notebook(range(3)):
    X_test.append(norm_data_x.iloc[i*n+i*256:i*n+n+i*256].  values)
    Y_test.append(i)
    indexes.append(data.index[i+n-1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [64]:
predictions = model.predict(np.asarray(X_test))
predictions = pd.DataFrame(predictions).rename(columns={0: 'prediction'})
actual=pd.DataFrame(Y_test).rename(columns={0: 'actual'})
final=pd.concat([predictions,actual],axis=1)
final['mae']=abs(final['prediction']-final['actual'])
final

2/2 [==============================] - 1s 85ms/step


,prediction,actual,mae
0,0.368380,0,0.368380
1,1.181365,1,0.181365
2,1.588598,2,0.411402
3,-0.019674,0,0.019674
4,0.614496,1,0.385504
5,1.649017,2,0.350983
6,0.091232,0,0.091232
7,1.133676,1,0.133676
8,2.017529,2,0.017529
9,0.097883,0,0.097883


In [65]:
import pickle
filename = 'finalized_color_model.sav'
pickle.dump(model, open(filename, 'wb'))